In [1]:
# The RSA.py file implements the RSA code from challenge 41, which I'll reuse here

from RSA import RSAClient, RSAServer

In [2]:
# First just verify everything is working...

message = b'Hello, world!'

server = RSAServer()
pubkey = server.GetPubkey()
client = RSAClient(**pubkey)

ciphertext = client.Encrypt(message)

assert server.Decrypt(ciphertext) == message

In [3]:
# Now make the oracle

class Oracle(RSAServer):
    def IsMessageEven(self, c):
        m = self.DecryptInt(c)
        return False if m%2 else True
    
server = Oracle()
pubkey = server.GetPubkey()
client = RSAClient(**pubkey)

ciphertext = client.Encrypt(message)
server.IsMessageEven(ciphertext)

False

In [4]:
# Now get the secret message

import base64

message = base64.decodebytes(b'VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ==')
ciphertext = client.Encrypt(message)

In [44]:
# The modulus is prime, which means it must be odd
# If I'm multiplying by powers of two... the result must be even
# UNTIL it wraps the modulus, at which point it MUST be odd

p = server.p
upper_bound = p
lower_bound = 0


for n in range(1024):
    multiplier = ( upper_bound + lower_bound ) // 2
    product = ciphertext * client.Encrypt(multiplier) % p
    
    if server.IsMessageEven(product):
        # We have not yet wrapped the modulus
        # So the multiplier is too small
        lower_bound = multiplier
    else:
        # We have wrapped the modulus
        # so the multiplier is too big
        upper_bound = multiplier
        
print(multiplier)
    

53855340626974444514605165184933525584662217621281995159088968472495597531686315540337196346707476232706118722799640697425016683899614948273444003990638230020163322872938265799565877302449470723628038496529885699086272681545862091668395197683896741423059026890295444880855538404776155142341645012052094959689


In [45]:
p//multiplier

1